In [23]:
import matplotlib.pyplot as plt
import ipywidgets as widgets
import seaborn as sns
import pandas as pd
import numpy as np

from matplotlib.animation import FuncAnimation
from ipywidgets import interact

sns.set_theme()

%matplotlib widget

Na Regressão Linear, é tentado aproximar a relação entre as variáveis independentes $x$ e as dependentes $y$ através de uma equação linear, do tipo

$$ \hat{y} = \beta_1 \cdot x + \beta_0 $$

onde $\hat{y}$ é a resposta calculada e $\hat{y} \approx y$, $x$ é a variável independente e os coeficientes $\beta_0$ e $\beta_1$ são os coeficientes linear e angular da reta.

Veja no exemplo abaixo:

In [24]:
df = pd.read_csv(r'.\Recursos\08-Linear-Regression-Models\Advertising.csv')
df['total-spend'] = df.iloc[:,:-1].sum(axis=1)
df.head()

,TV,radio,newspaper,sales,total-spend
0,230.1,37.8,69.2,22.1,337.1
1,44.5,39.3,45.1,10.4,128.9
2,17.2,45.9,69.3,9.3,132.4
3,151.5,41.3,58.5,18.5,251.3
4,180.8,10.8,58.4,12.9,250.0


In [ ]:
X = df['total-spend'].head(25).values
y = df['sales'].head(25).values
y_ = 0.005 * X + y.mean()

residual = y - y_
error = residual**2

fig, ax = plt.subplots(1, 2, figsize=(10,5))

ax[0].scatter(X, y, edgecolor='black', label=r'$y$')

ax[1].scatter(X, y, edgecolor='black', label=r'$y$')
ax[1].plot(X, y_, color='red', linewidth=1, label=r'$\hat{y}$')
ax[1].vlines(X, ymin=y_, ymax=y, linestyle='--', color='orange', label=r'$y - \hat{y}$')

plt.legend()
fig.tight_layout()

Contudo, como estimar os valores dos coeficientes $\beta_0$ e $\beta_1$ da reta que melhor descreve o problema?

A partir da relação estabelecida e da análise do gráfico, temos

$$ y_i = \beta_0 + \beta_1 \cdot x_i + \epsilon_i $$

$$ \hat{y}_i = \beta_0 + \beta_1 \cdot x_i $$

$$ y_i = \hat{y}_i + \epsilon_i $$

$$ y_i - \hat{y}_i = \epsilon_i $$

$$ \sum_{i=1}^n y_i - \hat{y}_i = \sum_{i=1}^n \epsilon_i $$

Portanto, a reta que melhor descreve o problema é aquele que minimiza a soma das diferenças entre os valores calculados e reais, ou seja, minimiza a soma dos erros $\epsilon_i$. Os valores de $\epsilon_i$ podem ser negativos e positivos. Dessa forma, ao invés da soma absoluta, é minimizado o erro médio quadrático (MSE):

$$ \frac{1}{n} \sum_{i=1}^n (y_i - \hat{y}_i)^2 = \frac{1}{n} \sum_{i=1}^n \epsilon_i^2 $$

In [ ]:
y_min = y.min() - 5
y_max = y.max() + 5

a_min = (y_min - y_max) / (X.max() - X.min())
a_max = - a_min

m = 25
a = np.linspace(a_min, a_max, m)
b = np.linspace(y_min, y_max, m)

A, B = np.meshgrid(a,b)

error_surface = ((y - (A.reshape(-1,1) * X.T + B.reshape(-1,1)))**2).mean(axis=1).reshape(m, m)

y_min, y_max = y.min(), y.max()

fig = plt.figure(figsize=(14,5))
# plt.subplots_adjust(bottom=0.25)

ax1 = fig.add_subplot(131)
ax2 = fig.add_subplot(132)
ax3 = fig.add_subplot(133, projection='3d')

ax1.scatter(X, y, edgecolor='black')
ax1_line, = ax1.plot(X, y_, color='red', linewidth=1)
ax1_vlines = ax1.vlines(X, ymin=y_, ymax=y, linestyle='--', color='orange')
ax1.set_ylim(y.min() - 5, y.max() + 5)
ax1.set_xlabel('x')
ax1.set_ylabel('y')

ax2.contourf(A, B, error_surface, 100, cmap='coolwarm')
ax2_scatter = ax2.scatter([], [], ec='black', s=80, color='yellow')
ax2.set_xlabel(r'$\beta_1$')
ax2.set_ylabel(r'$\beta_0$')

ax3.plot_surface(A, B, error_surface, cmap='coolwarm', zorder=0, linewidth=0)
ax3_line, = ax3.plot([], [], [], marker='o', color='yellow', zorder=2)
ax3.set_xlabel(r'$\beta_1$')
ax3.set_ylabel(r'$\beta_0$')
ax3.set_zlabel('MSE')

def update(a, b):

    y_ = a * X + b
    error = ((y - y_)**2).mean()
    
    ax1_line.set_ydata([y_])
    ax1_vlines.set_segments([[[xi, y_i], [xi, yi]] for xi, yi, y_i in zip(X, y, y_)])

    ax2_scatter.set_offsets([a, b])

    ax3_line.set_data([a], [b])
    ax3_line.set_3d_properties(error)

interact(
    update,
    a=widgets.FloatSlider(value=a[m//2], min=a[0], max=a[-1], step=1e-4, readout=True, readout_format='.4f'),
    b=widgets.FloatSlider(value=b[m//2], min=b[0], max=b[-1], step=1e-2, readout=True, readout_format='.2f')
)

fig.tight_layout()

Fazendo o erro médio quadrático em função dos parâmetros, temos

$$ J(\beta_0, \beta_1) = \sum_{i=1}^{m} (y_i - \hat{y_i})^2 $$

$$ = \sum_{i=1}^{m} [y_i - (\beta_0 + \beta_1 \cdot x_i)]^2 $$

$$ = \sum_{i=1}^{m} y_i^2 - 2 y_i (\beta_0 + \beta_1 \cdot x_i) + (\beta_0 + \beta_1 \cdot x_i)^2 $$

$$ = \sum_{i=1}^{m} y_i^2 -2 y_i \beta_0 - 2 y_i \beta_1 x + \beta_0^2 + 2 \beta_0 \beta_1 x + \beta_1^2 x^2$$

Para encontrar o erro $L$ mínimo, calcula-se a derivada do erro com relação aos parâmetros $\beta_0$ e $\beta_1$ e as iguala a zero:

$$ \frac{\partial}{\partial \beta_0} \sum_{i=1}^{m} (y_i - \hat{y}_i)^2 = 0 = \sum_{i=1}^{m} -2 y_i + 2 \beta_0 + 2 \beta_1 x_i $$

$$ \frac{\partial}{\partial \beta_1} \sum_{i=1}^{m} (y_i - \hat{y}_i)^2 = 0 = \sum_{i=1}^{m} -2 y_i x_i + 2 \beta_0 x_i + 2 \beta_1 x_i^2 $$

Tomando a primeira equação e isolando o parâmetro $\beta_0$, temos

$$ \sum_{i=1}^{m} y_i = \sum_{i=1}^{m} \beta_0 + \sum_{i=1}^{m} \beta_1 x_i $$

$$ \sum_{i=1}^{m} y_i = m \beta_0 + \beta_1 \sum_{i=1}^{m} x_i $$

$$ \beta_0 = \frac{1}{m} \left(\sum_{i=1}^{m} y_i - \beta_1 \sum_{i=1}^{m} x_i \right) $$

$$ \beta_0 = \overline{y} - \beta_1 \overline{x} $$

Tomando a segunda equação, isolando o parâmetro $\beta_1$ e substituindo o valor de $\beta_0$ encontrado anteriormente, temos

$$ \sum_{i=1}^{m} y_i x_i = \sum_{i=1}^{m} \beta_0 x_i + \sum_{i=1}^{m} \beta_1 x_i^2 $$

$$ \sum_{i=1}^{m} y_i x_i = (\overline{y} - \beta_1 \overline{x}) \sum_{i=1}^{m} x_i + \beta_1 \sum_{i=1}^{m} x_i^2 $$

$$ \sum_{i=1}^{m} y_i x_i = \overline{y} \sum_{i=1}^{m} x_i - \beta_1 \overline{x} \sum_{i=1}^{m} x_i + \beta_1 \sum_{i=1}^{m} x_i^2 $$

$$ \sum_{i=1}^{m} x_i y_i = \overline{y} \sum_{i=1}^{m} x_i + \beta_1 \left(\sum_{i=1}^{m} x_i^2 - \overline{x} \sum_{i=1}^{m} x_i \right) $$

$$ \beta_1 = \frac{\sum_{i=1}^{m} x_i y_i - \overline{y} \sum_{i=1}^{m} x_i}{\sum_{i=1}^{m} x_i^2 - \overline{x} \sum_{i=1}^{m} x_i} $$

$$ \beta_1 = \frac{\sum_{i=1}^{m} y_i x_i - m \overline{y} \cdot \overline{x}}{\sum_{i=1}^{m} x_i^2 - m \overline{x}^2} $$

$$ \beta_1 = \frac{\overline{y x} - \overline{y} \cdot \overline{x}}{\overline{x^2} - \overline{x}^2} $$

In [ ]:
a0 = ((X * y).mean() - X.mean() * y.mean()) / ((X**2).mean() - X.mean()**2)
b0 = y.mean() - a0 * X.mean()

print(f'beta0 = {round(b0, 3)} | beta1 = {round(a0, 3)}')

y_ = a0 * X + b0
error = ((y - y_)**2).mean()

error_surface = ((y - (A.reshape(-1,1) * X.T + B.reshape(-1,1)))**2).mean(axis=1).reshape(m, m)

fig = plt.figure(figsize=(14,5))
ax1 = fig.add_subplot(131)
ax2 = fig.add_subplot(132)
ax3 = fig.add_subplot(133, projection='3d') 

ax1.scatter(X, y, edgecolor='black')
ax1.plot(X, y_, color='red', linewidth=1)
ax1.vlines(X, ymin=y_, ymax=y, linestyle='--', color='orange')
ax1.set_xlabel(r'$x$')
ax1.set_ylabel(r'$y$')

ax2.contourf(A, B, error_surface, 100, cmap='coolwarm')
ax2.scatter(a0, b0, ec='black', s=80, color='yellow')
ax2.set_xlabel(r'$\beta_1$')
ax2.set_ylabel(r'$\beta_0$')

ax3.plot_surface(A, B, error_surface, cmap='coolwarm', zorder = 0, linewidth=0)
ax3.plot(a0, b0, error, marker='o', color='yellow', zorder = 2)
ax3.set_xlabel(r'$\beta_1$')
ax3.set_ylabel(r'$\beta_0$')
ax3.set_zlabel(r'MSE')

fig.tight_layout()
plt.show()

Para problemas com múltiplas variáveis independentes, encontrar a solução analítica para o problema de mínimos quadrados se torna praticamente inviável. Para contornar este problema, são utilizados dois métodos: $\textbf{Gradiente Descendente}$ e $\textbf{matriz pseudo-inversa}$.

Para facilitar a implementação computacional, é comum a utilização da notação matricial, assim

$$ \hat{Y} = X \beta $$

onde $\hat{Y}$ é o vetor correspondente às respostas calculadas, $\beta$ o vetor correspondente aos coeficientes e $X$ a matriz correspondente às variáveis independentes.

$$ \hat{Y} = \begin{bmatrix}
\hat{y}_1 \\
\hat{y}_2 \\
\hat{y}_3 \\
\dots \\
\hat{y}_m
\end{bmatrix}, $$

$$ \beta = \begin{bmatrix}
\beta_0 \\
\beta_1 \\
\beta_2 \\
\dots \\
\beta_n
\end{bmatrix}, $$

$$ X = \begin{bmatrix}
1 & x_{1,2} & x_{1,3} & \dots & x_{1,n} \\
1 & x_{2,2} & x_{2,3} & \dots & x_{2,n} \\
1 & x_{3,2} & x_{3,3} & \dots & x_{3,n} \\
\dots & \dots & \dots & \dots & \dots \\
1 & x_{m,2} & x_{m,3} & \dots & x_{m,n}
\end{bmatrix}. $$

$$ \hat{Y} = X \beta = \begin{bmatrix}
\hat{y}_1 \\
\hat{y}_2 \\
\hat{y}_3 \\
\dots \\
\hat{y}_m
\end{bmatrix} = \begin{bmatrix}
1 & x_{1,2} & x_{1,3} & \dots & x_{1,n} \\
1 & x_{2,2} & x_{2,3} & \dots & x_{2,n} \\
1 & x_{3,2} & x_{3,3} & \dots & x_{3,n} \\
\dots & \dots & \dots & \dots & \dots \\
1 & x_{m,2} & x_{m,3} & \dots & x_{m,n}
\end{bmatrix} \begin{bmatrix}
\beta_0 \\
\beta_1 \\
\beta_2 \\
\dots \\
\beta_n
\end{bmatrix} $$

#### Solução Matricial (Pseudo-Inversa)

Vimos anteriormente que 

$$ \hat{Y} = X \beta$$

Portanto,

$$ e = Y - \hat{Y} $$
$$ = Y - X \beta $$

$$ \text{MSE} = J(\beta) = e^T e $$
$$ = (Y - \hat{Y})^T (Y - \hat{Y}) $$
$$ = (Y - X \beta)^T (Y - X \beta) $$
$$ = \left(Y^T - (X \beta)^T \right) (Y - X \beta) $$
$$ = (Y^T - \beta^T X^T) (Y - X \beta) $$
$$ = Y^T Y - Y^T X \beta - \beta^T X^T Y + \beta^T X^T X \beta $$

Para minimizar o $\text{J}(\beta)$, tomamos a derivada em relação a $\beta$ e igualamos a zero:

$$ \frac{\partial \text{MSE}}{\partial \beta} = 0 = 0 - X^T Y - X^T Y + 2 X^T X \beta$$
$$ = X^T X \beta - X^T Y $$

Assim, é obtido o sistema de equações normais:

$$ X^T X \beta = X^T Y $$

Para obter a solução da equação anterior, devemos multiplicar ambos os lados da igualdade pela inversa de $X^T X$:

$$ (X^T X)^{-1} X^T X \beta = (X^T X)^{-1} X^T Y$$

Como $(X^T X)^{-1} (X^T X) = I$ e $I \beta = \beta$, temos:

$$ \beta = (X^T X)^{-1} X^T Y$$

Vamos testar

In [ ]:
X = np.concatenate([np.ones(len(X)).reshape(-1,1), X.reshape(-1,1)], axis=1)

beta = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)

print(rf'beta0 = {round(b0, 3)} | beta1 = {round(a0, 3)}')

y_ = X.dot(beta)
error = ((y - y_)**2).mean()

error_surface = ((y - (A.reshape(-1,1) * X[:,1].T + B.reshape(-1,1)))**2).mean(axis=1).reshape(m, m)

fig = plt.figure(figsize=(14,5))
ax1 = fig.add_subplot(131)
ax2 = fig.add_subplot(132)
ax3 = fig.add_subplot(133, projection='3d') 

ax1.scatter(X[:,1], y, edgecolor='black')
ax1.plot(X[:,1], y_, color='red', linewidth=1)
ax1.vlines(X[:,1], ymin=y_, ymax=y, linestyle='--', color='orange')
ax1.set_xlabel(r'$x$')
ax1.set_ylabel(r'$y$')

ax2.contourf(A, B, error_surface, 100, cmap='coolwarm')
ax2.scatter(a0, b0, ec='black', s=80, color='yellow')
ax2.set_xlabel(r'$\beta_1$')
ax2.set_ylabel(r'$\beta_0$')

ax3.plot_surface(A, B, error_surface, cmap='coolwarm', zorder = 0, linewidth=0)
ax3.plot(a0, b0, error, marker='o', color='yellow', zorder = 2)
ax3.set_xlabel(r'$\beta_1$')
ax3.set_ylabel(r'$\beta_0$')
ax3.set_zlabel(r'MSE')

fig.tight_layout()
plt.show()

#### Gradiente Descendente

Generalizando o cálulo das derivadas, temos

$$ \frac{\partial J(\beta)}{\partial \beta_k} = \frac{2}{m} \sum_{i=1}^m \left(y_i - \sum_{j=0}^n \beta_j x_{i,j} \right) (-x_{i,k}) $$

Podemos usar o gradiente $\nabla$ para expressar a derivada da função custo com relação a todos os coeficientes $\beta$, de acordo com a expressão $\nabla_{\beta}J$. Dessa forma,

$$ \nabla_{\beta}J = \begin{bmatrix}
\frac{\partial J}{\partial \beta_0} &
\frac{\partial J}{\partial \beta_1} &
\dots &
\frac{\partial J}{\partial \beta_n}
\end{bmatrix} $$

$$ \nabla_{\beta}J = \begin{bmatrix}
\frac{1}{m} \sum_{i=1}^m \left(y_i - \sum_{j=0}^n \beta_j x_{i,j} \right) (-x_{i,0}) \\
\frac{1}{m} \sum_{i=1}^m \left(y_i - \sum_{j=0}^n \beta_j x_{i,j} \right) (-x_{i,1}) \\
\dots \\
\frac{1}{m} \sum_{i=1}^m \left(y_i - \sum_{j=0}^n \beta_j x_{i,j} \right) (-x_{i,n})
\end{bmatrix}^T $$

Dessa forma, é possível atualizar os parâmetros de acordo com o gradiente na direção de mínimo

$$ \beta_{t+1} = \beta_t - \alpha \cdot \nabla_{\beta}J $$

onde $\beta_{t+1}$ é o parâmetro atualizado, $\beta_t$ o parâmetro atual, $\alpha$ uma constante chamada $\textit{learning rate}$, que controla a intensidade da atualização, e $\nabla_{\beta}J$ o vetor gradiente da função custo.

In [ ]:
X = df.iloc[:25, -1].values.reshape(-1,1)
X = (X - np.mean(X)) / np.std(X)
X = np.concatenate([np.ones(len(X)).reshape(-1,1), X], axis=1)

y = df.iloc[:25, -2].values + 40

beta = np.asarray([0., 0.])

history = {
    'loss' : [],
    'beta' : [],
    'y_pred' : []
}

m = len(y)
lr = 1e-01
epochs = 50

for _ in range(epochs):
    y_pred = X.dot(beta)
    error = y_pred - y
    mse = error.T.dot(error) / m
    gradient = -2 * X.T.dot(error) / m
    beta += lr * gradient

    history['loss'].append(mse)
    history['beta'].append(beta.round(2))
    history['y_pred'].append(y_pred)

beta0 = np.linspace(0, 100, 25)
beta1 = np.linspace(-100, 100, 25)
beta0, beta1 = np.meshgrid(beta0, beta1)
beta_surface = np.concatenate([beta0.reshape(-1,1), beta1.reshape(-1,1)], axis=1)

error_surface = y.reshape(-1,1) - X.dot(beta_surface.T)
mse_surface = np.asarray([line.T.dot(line) for line in error_surface.T]).reshape(25,25) / m

fig = plt.figure(figsize=(16,10))
ax1 = fig.add_subplot(221)
ax2 = fig.add_subplot(222)
ax3 = fig.add_subplot(235, projection='3d')

ax1.plot(history['loss'], color='black', label='Custo (não normalizado)', zorder=0)
ax1_scatter = ax1.scatter([], [], c='yellow', ec='black', zorder=1)
ax1.set_title('Curva de Aprendizado')
ax1.set_xlabel('Época')
ax1.set_ylabel('MSE')
ax1.grid(False)
ax1.legend()

ax2.scatter(X[:,1], y, color='orange', ec='black', label=r'Observado ($y$)', zorder=1)
ax2_line, = ax2.plot(X[:,1], X.dot(beta), color='black', linewidth=1, label=r'Calculado ($X\beta$)', zorder=0)
ax2_annot = ax2.annotate(text='', xy=(-3,80))
ax2.set_xlabel('X')
ax2.set_ylabel('y')
ax2.set_ylim(0, 100)
ax2.set_xlim(-5,5)
ax2.grid(False)
ax2.legend()

ax3.plot_surface(beta0, beta1, mse_surface, cmap='coolwarm', lw=0, zorder=0)
ax3_line, = ax3.plot([], [], [], marker='o', c='yellow', zorder=2)

def update(epoch):
    ax1_scatter.set_offsets([epoch, history['loss'][epoch]])

    ax2_line.set_ydata(history['y_pred'][epoch])
    ax2_annot.set_text(rf'$\beta$ = {history["beta"][epoch]}')
    ax2.set_title(f'Distribuição e Modelo Ajustado \nÉpoca: {epoch}')

    ax3_line.set_data([history['beta'][epoch][0]], [history['beta'][epoch][1]])
    ax3_line.set_3d_properties([history['loss'][epoch]])

# fig.tight_layout()
# plt.show()

interact(update, epoch=widgets.IntSlider(value=0, min=0, max=epochs-1))